In [1]:
from nilearn import datasets
from preprocess_data import Reader
import os
import shutil
import sys
import pandas as pd
import numpy as np
import deepdish as dd
import warnings
import os.path as osp
from pathlib import Path

/neuro/labs/grantlab/users/arafat.hussain/bnt/lib64/python3.6/site-packages/nilearn/__init__.py:69: FutureWarning: Python 3.6 support is deprecated and will be removed in release 0.10 of Nilearn. Consider switching to Python 3.8 or 3.9.
  _python_deprecation_warnings()


In [2]:
def str2bool(v):
    if isinstance(v, bool):
        return v
    if v.lower() in ('yes', 'true', 't', 'y', '1'):
        return True
    elif v.lower() in ('no', 'false', 'f', 'n', '0'):
        return False
    else:
        raise argparse.ArgumentTypeError('Boolean value expected.')

#### Fetch Data

In [ ]:
root_folder = '/home/ch225256/Data'
data_folder = os.path.join(root_folder, 'ABIDE_pcp/cpac/filt_noglobal/')
if not os.path.exists(data_folder):
    os.makedirs(data_folder)

pipeline = 'cpac'
atlas = 'cc200'
download = True
id_file_path = 'subject_IDs.txt'

# Files to fetch

files = ['rois_' + atlas]

# Download database files
if download == True:
    abide = datasets.fetch_abide_pcp(data_dir=root_folder, pipeline=pipeline,
                                        band_pass_filtering=True, global_signal_regression=False, derivatives=files,
                                        quality_checked=False)

#### Subject-wise foldering and extracting correlation 

In [ ]:
root_folder = '/home/ch225256/Data'
id_file_path = 'subject_IDs.txt'
reader = Reader(root_folder, id_file_path)
data_folder = os.path.join(root_folder, 'ABIDE_pcp/cpac/filt_noglobal/')

pipeline = 'cpac'
atlas = 'cc200'
files = ['rois_' + atlas]
filemapping = {'func_preproc': 'func_preproc.nii.gz',
                files[0]: files[0] + '.1D'}

#phenotype_file = os.path.join(root_folder, "ABIDE_pcp/Phenotypic_V1_0b_preprocessed1.csv")
#phenotype_df = pd.read_csv(phenotype_file, index_col=0)

temp = '/home/ch225256/Data/subject_IDs.txt'
subject_IDs = np.genfromtxt(temp, dtype=str)

# Create a folder for each subject
for s, fname in zip(subject_IDs, reader.fetch_filenames(subject_IDs, files[0], atlas)):
    subject_folder = os.path.join(data_folder, s)
    if not os.path.exists(subject_folder):
        os.mkdir(subject_folder)

    # Get the base filename for each subject
    base = fname.split(files[0])[0]

    # Move each subject file to the subject folder
    for fl in files:
        if not os.path.exists(os.path.join(subject_folder, base + filemapping[fl])):
            shutil.move(base + filemapping[fl], subject_folder)

time_series = reader.get_timeseries(subject_IDs, atlas)

# Compute and save connectivity matrices
reader.subject_connectivity(time_series, subject_IDs, atlas, 'correlation')
reader.subject_connectivity(time_series, subject_IDs, atlas, 'partial correlation')

#### Correlation matrix generation

In [2]:

root_path = '/home/ch225256/Data'
data_folder = os.path.join(root_path, 'ABIDE_pcp/cpac/filt_noglobal/')

params = dict()

params['seed'] = 123  # seed for random initialization
id_file_path = 'subject_IDs.txt'

# Algorithm choice
params['atlas'] = 'cc200'  # Atlas for network construction
atlas = 'cc200'  # Atlas for network construction (node definition)

reader = Reader(root_path, id_file_path)
# Get subject IDs and class labels
temp = '/home/ch225256/Data/subject_IDs.txt'
subject_IDs = np.genfromtxt(temp, dtype=str)
labels = reader.get_subject_score(subject_IDs, score='DX_GROUP')

# Number of subjects and classes for binary classification
num_classes = 2
num_subjects = len(subject_IDs)
params['n_subjects'] = num_subjects

# Initialise variables for class labels and acquisition sites
# 1 is autism, 2 is control
y_data = np.zeros([num_subjects, num_classes]) # n x 2
y = np.zeros([num_subjects, 1]) # n x 1

# Get class labels for all subjects
for i in range(num_subjects):
    y_data[i, int(labels[subject_IDs[i]]) - 1] = 1
    y[i] = int(labels[subject_IDs[i]])

# Compute feature vectors (vectorized connectivity networks)
fea_corr = reader.get_networks(subject_IDs, iter_no='', kind='correlation', atlas_name=atlas) #(1035, 200, 200)
fea_pcorr = reader.get_networks(subject_IDs, iter_no='', kind='partial correlation', atlas_name=atlas) #(1035, 200, 200)

if not os.path.exists(os.path.join(data_folder,'raw')):
    os.makedirs(os.path.join(data_folder,'raw'))
for i, subject in enumerate(subject_IDs):
    dd.io.save(os.path.join(data_folder,'raw',subject+'.h5'),{'corr':fea_corr[i],'pcorr':fea_pcorr[i],'label':(y[i]-1)})

#### Final Data Generation

In [26]:
root_path = '/home/ch225256/Data'
data_dir =  os.path.join(root_path, 'ABIDE_pcp/cpac/filt_noglobal/raw')
timeseires = os.path.join(root_path, 'ABIDE_pcp/cpac/filt_noglobal/')

meta_file = os.path.join(root_path, "ABIDE_pcp/Phenotypic_V1_0b_preprocessed1.csv")
meta_file = pd.read_csv(meta_file, header=0)

id2site = meta_file[["subject", "SITE_ID"]]

# pandas to map
id2site = id2site.set_index("subject")
id2site = id2site.to_dict()['SITE_ID']

times = []
piq = []
viq = []
fiq = []
labels = []
pcorrs = []
corrs = []
site_list = []

for f in os.listdir(data_dir):
    if osp.isfile(osp.join(data_dir, f)):
        fname = f.split('.')[0]
        site = id2site[int(fname)]
        
        fiq_ = meta_file.loc[meta_file['subject'] == int(fname), 'FIQ'].values[0]
        viq_ = meta_file.loc[meta_file['subject'] == int(fname), 'VIQ'].values[0]
        piq_ = meta_file.loc[meta_file['subject'] == int(fname), 'PIQ'].values[0]
        
        if np.isnan(fiq_) or np.isnan(viq_) or np.isnan(piq_):
            continue
        elif fiq_ > 200 or fiq_ < 20:
            continue
        elif viq_ > 200 or viq_ < 20:
            continue
        elif piq_ > 200 or piq_ < 20:
            continue
        
        files = os.listdir(osp.join(timeseires, fname))

        file = list(filter(lambda x: x.endswith("1D"), files))[0]

        time = np.loadtxt(osp.join(timeseires, fname, file), skiprows=0).T

        if time.shape[1] < 100:
            continue

        temp = dd.io.load(osp.join(data_dir,  f))
        pcorr = temp['pcorr'][()]

        pcorr[pcorr == float('inf')] = 0

        att = temp['corr'][()]

        att[att == float('inf')] = 0

        label = temp['label']

        times.append(time[:,:100])
        labels.append(label[0])
        fiq.append(fiq_)
        viq.append(viq_)
        piq.append(piq_)
        corrs.append(att)
        pcorrs.append(pcorr)
        site_list.append(site)

np.save(Path(root_path)/'ABIDE_pcp/abide.npy', {'timeseires': np.array(times), "label": np.array(labels), "fiq": np.array(fiq), "viq": np.array(viq), "piq": np.array(piq), "corr": np.array(corrs),"pcorr": np.array(pcorrs), 'site': np.array(site_list)})

### New dataset with residual scores

In [3]:
root_path = '/neuro/labs/grantlab/users/arafat.hussain/Data'
data_dir =  os.path.join(root_path, 'ABIDE_pcp/cpac/filt_noglobal/raw')
timeseires = os.path.join(root_path, 'ABIDE_pcp/cpac/filt_noglobal/')

meta_file = os.path.join(root_path, "ABIDE_pcp/Phenotypic_V1_0b_preprocessed1.csv")
meta_file = pd.read_csv(meta_file, header=0)

id2site = meta_file[["subject", "SITE_ID"]]

# pandas to map
id2site = id2site.set_index("subject")
id2site = id2site.to_dict()['SITE_ID']

In [4]:
# assinging numbers to sex
sex = meta_file["SEX"]
sex[sex == "M"] = 1
sex[sex == "F"] = 2
meta_file["SEX"] = sex

# dropping IQ of NAN rows
meta_file=meta_file[meta_file["FIQ"].notna()].reset_index(drop=True)
meta_file=meta_file[meta_file["VIQ"].notna()].reset_index(drop=True)
meta_file=meta_file[meta_file["PIQ"].notna()].reset_index(drop=True)

# dropping IQ of negative values
meta_file=meta_file[meta_file["FIQ"]>=0].reset_index(drop=True)
meta_file=meta_file[meta_file["PIQ"]>=0].reset_index(drop=True)
meta_file=meta_file[meta_file["VIQ"]>=0].reset_index(drop=True)
meta_file

,Unnamed: 0,Unnamed: 0.1,SUB_ID,X,subject,SITE_ID,FILE_ID,DX_GROUP,DSM_IV_TR,AGE_AT_SCAN,...,qc_notes_rater_1,qc_anat_rater_2,qc_anat_notes_rater_2,qc_func_rater_2,qc_func_notes_rater_2,qc_anat_rater_3,qc_anat_notes_rater_3,qc_func_rater_3,qc_func_notes_rater_3,SUB_IN_SMP
0,0,1,50002,1,50002,PITT,no_filename,1,1,16.77,...,NaN,OK,NaN,fail,ic-parietal-cerebellum,OK,NaN,fail,ERROR #24,1
1,1,2,50003,2,50003,PITT,Pitt_0050003,1,1,24.45,...,NaN,OK,NaN,OK,NaN,OK,NaN,OK,NaN,1
2,2,3,50004,3,50004,PITT,Pitt_0050004,1,1,19.09,...,NaN,OK,NaN,OK,NaN,OK,NaN,OK,NaN,1
3,3,4,50005,4,50005,PITT,Pitt_0050005,1,1,13.73,...,NaN,OK,NaN,maybe,ic-parietal-cerebellum,OK,NaN,OK,NaN,0
4,4,5,50006,5,50006,PITT,Pitt_0050006,1,1,13.37,...,NaN,OK,NaN,maybe,ic-parietal slight,OK,NaN,OK,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
863,1079,1080,51493,1080,51493,CALTECH,Caltech_0051493,2,0,29.20,...,NaN,OK,NaN,maybe,Ic-parietal-minor,OK,NaN,OK,NaN,0
864,1102,1103,51578,1103,51578,SBL,SBL_0051578,1,2,33.00,...,NaN,OK,NaN,maybe,ic-cerebellum-temporal_lobe,OK,NaN,OK,NaN,0
865,1105,1106,51581,1106,51581,SBL,SBL_0051581,1,3,64.00,...,NaN,OK,NaN,fail,ic-parietal-cerebellum,OK,NaN,OK,NaN,0
866,1107,1108,51583,1108,51583,SBL,SBL_0051583,1,2,35.00,...,NaN,OK,NaN,OK,ic-cerebellum-temporal_lobe,OK,NaN,OK,NaN,0


In [5]:
site = meta_file[["SITE_ID"]]
site[site == "CALTECH"] = 1
site[site == "CMU"] = 2
site[site == "LEUVEN_1"] = 3
site[site == "NYU"] = 4
site[site == "PITT"] = 5
site[site == "SBL"] = 6
site[site == "SDSU"] = 7
site[site == "STANFORD"] = 8
site[site == "TRINITY"] = 9
site[site == "UCLA_1"] = 10
site[site == "UCLA_2"] = 11
site[site == "UM_1"] = 12
site[site == "UM_2"] = 13
site[site == "USM"] = 14
site[site == "YALE"] = 15

meta_file["SITE_ID_no"] = site
meta_file

,Unnamed: 0,Unnamed: 0.1,SUB_ID,X,subject,SITE_ID,FILE_ID,DX_GROUP,DSM_IV_TR,AGE_AT_SCAN,...,qc_anat_rater_2,qc_anat_notes_rater_2,qc_func_rater_2,qc_func_notes_rater_2,qc_anat_rater_3,qc_anat_notes_rater_3,qc_func_rater_3,qc_func_notes_rater_3,SUB_IN_SMP,SITE_ID_no
0,0,1,50002,1,50002,PITT,no_filename,1,1,16.77,...,OK,NaN,fail,ic-parietal-cerebellum,OK,NaN,fail,ERROR #24,1,5
1,1,2,50003,2,50003,PITT,Pitt_0050003,1,1,24.45,...,OK,NaN,OK,NaN,OK,NaN,OK,NaN,1,5
2,2,3,50004,3,50004,PITT,Pitt_0050004,1,1,19.09,...,OK,NaN,OK,NaN,OK,NaN,OK,NaN,1,5
3,3,4,50005,4,50005,PITT,Pitt_0050005,1,1,13.73,...,OK,NaN,maybe,ic-parietal-cerebellum,OK,NaN,OK,NaN,0,5
4,4,5,50006,5,50006,PITT,Pitt_0050006,1,1,13.37,...,OK,NaN,maybe,ic-parietal slight,OK,NaN,OK,NaN,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
863,1079,1080,51493,1080,51493,CALTECH,Caltech_0051493,2,0,29.20,...,OK,NaN,maybe,Ic-parietal-minor,OK,NaN,OK,NaN,0,1
864,1102,1103,51578,1103,51578,SBL,SBL_0051578,1,2,33.00,...,OK,NaN,maybe,ic-cerebellum-temporal_lobe,OK,NaN,OK,NaN,0,6
865,1105,1106,51581,1106,51581,SBL,SBL_0051581,1,3,64.00,...,OK,NaN,fail,ic-parietal-cerebellum,OK,NaN,OK,NaN,0,6
866,1107,1108,51583,1108,51583,SBL,SBL_0051583,1,2,35.00,...,OK,NaN,OK,ic-cerebellum-temporal_lobe,OK,NaN,OK,NaN,0,6


In [8]:
print(max(meta_file["FIQ"]), min(meta_file["FIQ"]))
print(max(meta_file["PIQ"]), min(meta_file["PIQ"]))
print(max(meta_file["VIQ"]), min(meta_file["VIQ"]))

148.0 41.0
157.0 37.0
180.0 42.0


In [6]:
from sklearn.linear_model import LinearRegression
X = meta_file[["DX_GROUP","AGE_AT_SCAN","SEX","SITE_ID_no"]]
y = meta_file[["FIQ"]]
reg = LinearRegression().fit(X, y)
print(reg.coef_, reg.intercept_)
meta_file["FIQ_res"] = y - reg.predict(X)
print(max(meta_file["FIQ_res"]), min(meta_file["FIQ_res"]))

[[ 5.39044046  0.1202752  -1.05296575 -0.60005726]] [104.16463291]
45.09204766237907 -60.47616751628318


In [7]:
from sklearn.linear_model import LinearRegression
X = meta_file[["DX_GROUP","AGE_AT_SCAN","SEX","SITE_ID_no"]]
y = meta_file[["PIQ"]]
reg = LinearRegression().fit(X, y)
print(reg.coef_, reg.intercept_)
meta_file["PIQ_res"] = y - reg.predict(X)
print(max(meta_file["PIQ_res"]), min(meta_file["PIQ_res"]))

[[ 2.88842957  0.13322843 -1.81951583 -0.59632508]] [107.08940302]
52.37712308727383 -64.40105140949484


In [8]:
from sklearn.linear_model import LinearRegression
X = meta_file[["DX_GROUP","AGE_AT_SCAN","SEX","SITE_ID_no"]]
y = meta_file[["VIQ"]]
reg = LinearRegression().fit(X, y)
print(reg.coef_, reg.intercept_)
meta_file["VIQ_res"] = y - reg.predict(X)
print(max(meta_file["VIQ_res"]), min(meta_file["VIQ_res"]))

[[ 6.6291643   0.07322294 -0.24249323 -0.45581154]] [100.84912361]
77.5602927521889 -59.60094227588064


In [12]:
meta_file.to_csv('test.csv', index=False)

In [37]:
print(max(meta_file["PIQ"]), min(meta_file["PIQ"]))
print(max(meta_file["PIQ_res"]), min(meta_file["PIQ_res"]))

157.0 37.0
52.377123087273844 -64.40105140949483


In [ ]:
import statistics
times = []
piq = []
viq = []
fiq = []
piq_r = []
viq_r = []
fiq_r = []
labels = []
pcorrs = []
corrs = []
site_list = []
age = []

count = 0
male = 0
ASD = 0

for f in os.listdir(data_dir):
    if osp.isfile(osp.join(data_dir, f)):
        fname = f.split('.')[0]
        site = id2site[int(fname)]
        if not meta_file['subject'].eq(int(fname)).any():
            continue
        
        fiq_ = meta_file.loc[meta_file['subject'] == int(fname), 'FIQ'].values[0]
        viq_ = meta_file.loc[meta_file['subject'] == int(fname), 'VIQ'].values[0]
        piq_ = meta_file.loc[meta_file['subject'] == int(fname), 'PIQ'].values[0]
        fiq_res = meta_file.loc[meta_file['subject'] == int(fname), 'FIQ_res'].values[0]
        viq_res = meta_file.loc[meta_file['subject'] == int(fname), 'VIQ_res'].values[0]
        piq_res = meta_file.loc[meta_file['subject'] == int(fname), 'PIQ_res'].values[0]
        sex = meta_file.loc[meta_file['subject'] == int(fname), 'SEX'].values[0]
        diagnosis = meta_file.loc[meta_file['subject'] == int(fname), 'DX_GROUP'].values[0]
        age_ = meta_file.loc[meta_file['subject'] == int(fname), 'AGE_AT_SCAN'].values[0]
        
        files = os.listdir(osp.join(timeseires, fname))

        file = list(filter(lambda x: x.endswith("1D"), files))[0]

        time = np.loadtxt(osp.join(timeseires, fname, file), skiprows=0).T

        if time.shape[1] < 100:
            continue

        temp = dd.io.load(osp.join(data_dir,  f))
        pcorr = temp['pcorr'][()]

        pcorr[pcorr == float('inf')] = 0

        att = temp['corr'][()]

        att[att == float('inf')] = 0

        label = temp['label']

        times.append(time[:,:100])
        labels.append(label[0])
        fiq.append(fiq_)
        viq.append(viq_)
        piq.append(piq_)
        fiq_r.append(fiq_res)
        viq_r.append(viq_res)
        piq_r.append(piq_res)
        corrs.append(att)
        pcorrs.append(pcorr)
        age.append(age_)
        
        if sex == 1: 
            male += 1
        if diagnosis == 1:
            ASD += 1
        count += 1

np.save(Path(root_path)/'ABIDE_pcp/abide_absolute_residual_scores_with_path.npy', {'timeseires': np.array(times), "label": np.array(labels), "fiq": np.array(fiq), "viq": np.array(viq), "piq": np.array(piq), "fiq_r": np.array(fiq_r), "viq_r": np.array(viq_r), "piq_r": np.array(piq_r), "corr": np.array(corrs),"pcorr": np.array(pcorrs), 'site': np.array(site_list)})
ageMean = statistics.mean(age)
ageSTD = statistics.stdev(age)
mini = min(age)
maxa = max(age)
print(f'Total: {count}, Male: {male}, ASD: {ASD}, Age mean: {ageMean}, Age std: {ageSTD}, Age Min: {mini}, Age Max: {maxa}')